In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import pickle




In [7]:
with open('datasets/teconer_helsinki_jan2018_df.pkl', 'rb') as f:
        df = pickle.load(f)
        # randomly sample 10K records from the dataset
        df = df.sample(n=10000, random_state=1234)

print(df.head())
# save the dataset to a csv file with the specified columns
df.to_csv('datasets/helsinki_10K.csv', columns=['UnixTime', 'Latitude', 'Longitude', 'Height', 'Speed', 'Direction', 'Ta', 'Tsurf', 'Hour', 'Day', 'TripID', 'Friction'], index=False)

           UnixTime           Timestamp   Latitude  Longitude  Height  Speed  \
1421940  1516627798 2018-01-22 13:29:58  60.191305  24.954031    28.0   22.9   
680113   1515065608 2018-01-04 11:33:28  60.194093  24.947437    34.0   31.6   
1244533  1516176645 2018-01-17 08:10:45  60.210397  25.077656    15.0    0.0   
1741382  1517388279 2018-01-31 08:44:39  60.196368  24.928772    27.0   10.0   
55348    1515691233 2018-01-11 17:20:33  60.204922  24.848886     3.0  100.3   

         Direction   Ta  Tsurf  Distance  ...     S3      S9    S10    S11  \
1421940      126.1 -2.1  -3.14    6699.0  ...  2.829   753.4  798.8  1.795   
680113       126.9  5.0   2.67   16780.0  ... -0.459   742.2  789.0  1.795   
1244533       61.3 -3.7  -4.11   18775.0  ...  0.410   759.7  804.6  1.810   
1741382      175.6 -2.0  -1.98   42414.0  ... -0.111   749.6  796.3  1.795   
55348        111.9 -0.7  -1.23  156360.0  ... -0.107  1015.1  674.6  1.807   

         Hour  Month  Day  VehicleID  TripID Frict